### Background substraction and pipette properties detection

This script takes as input: 
* a video of the chemotaxis experiment 

and returns as output:

 * the bckg substracted black and white video 
 * a dictionary with the pipette properties: countour lines, xy coordinates of the pipette entrance centers and inclination angle with respect to the horizontal    
 * a control image to check if the contour of the pipette is well detected
 
 
what if the pipette detection is wrong?

Check on the control image that the pipette detection is good: countour, entrance center position and angle (to check if the angle is right, check if the line traced in the center of the pipette passes from the entrance center). 
The detection can be bad for various reasons: 

* **check the binary image**: is the white profile of the pipette smooth? The pipette shape should be white and the background homogenously black. If there are white or black spots where there shouldn't, **try to change the threshold** in the conversion from greyscale to binary:  the parameter "threshold" is the number in the grey scale above which the color will be turned to white in the binary image. If there are white spots, increase the threshold, if there are black spots, decrease the threshold 

* **check the plateau size**: the pipette external thick border is detected as a wide white plateau in the binary image. **Increasing or reducing the plateau size** can be improve the detection of the contour 

* **Is the video short?** the fewer the frames, the worst are the outputs!


*Written by Medea Zanoli on 02/01/2021*

In [1]:
from PIL import Image, ImageEnhance, ImageSequence, ImageOps,  ImageChops, ImageFilter,  ImageDraw

from tkinter import Tk
from tkinter.filedialog import askdirectory, askopenfilename

import os
import glob

import matplotlib.pyplot as plt
%matplotlib inline 

import numpy as np

from progressbar import ProgressBar
import progressbar

import pandas as pd
import cv2 

import seaborn as sns

from scipy.signal import find_peaks
from scipy.optimize import curve_fit

from matplotlib import path

import pickle

### Import the video to process

In [2]:
imput_video = askopenfilename() 

### Manually select the folder where to save the outputs 

In [4]:
output_path =  askdirectory() 

### Chose the name under which the outouts will be saved 

In [7]:
# video name 
video_name = 'prova'

### Calculate background and pipette properties 

In [10]:
# IF THE PIPETTE DETECTION IS BAD...
# ...TRY TO CHANGE THE THRESHOLD!: thresh is the number in the grey scale above which there will be white in 
# the binary image. 
# If thresh is too low, blurry areas around the pipette can become parts of the pipette in the conversion to a binary image
threshold = 240
# IF THE PIPETTE DETECTION IS BAD...
# try change the plateau size 
plateau_size = [2, 7]# max and min number of pixels of the pipette internal contour   

# import video to process
cap= cv2.VideoCapture(imput_video)

# define a progress bar object
bar = ProgressBar(maxval=int(cap.get(cv2.CAP_PROP_FRAME_COUNT)))
bar.start()

# background computation
print('Extracting images and computing background...')

i=0
images_array = [] # here we save the video frames while computing brackground 
bckg_arr = np.zeros(np.shape(cap.read()[1])[0:2]) # initialize background matrix to zero, get the shape from the first file

while(cap.isOpened()):
    
    ret, frame = cap.read() 
    if ret == False:
        break
        
    # image processing
    im = Image.fromarray(frame, mode="RGB")
    im = im.convert("L") # convert to greyscale
    im = ImageOps.invert(im) # invert colors 
    im = ImageEnhance.Contrast(im).enhance(1.9) # increment contrast by .x % 
    
    # save the processed image
    im_arr = np.array(im, dtype=np.uint8)  # vectorize frame image in uint8 format and save it for later use
    images_array.append(im_arr) #save image in an array 
    
    # calculate background
    bckg_arr = im_arr.astype(float) + bckg_arr.astype(float) # sum as float <-- important not to screw up evrything
    i+=1 # frames counter to normalize the bckg 
    
    bar.update(i) # updat progressbar
    
    
bckg_arr = (bckg_arr/i).astype(np.uint8) # convert bckg to uint8
bckg_im = Image.fromarray(bckg_arr,mode="L")

bckg_im.save(output_path + "/" + video_name + "_bckg.pdf")

cap.release() # close imput video
cv2.destroyAllWindows()

print('bckg computation is done')

# prepare video to save processed and bckg subtracted images 
print('saving bckg subtracted images in a video...')

# prepare dimension for video file
height, width = np.shape(bckg_arr)
size = (width, height)
fourcc = cv2.VideoWriter_fourcc(*'XVID') # format: great mistery

# open an empty video file with the right dimensions
out = cv2.VideoWriter(output_path +'/'+ video_name + '.avi', # directory and name of the video  
                      fourcc,  
                      30,  # frame per second in output video
                      size, # framesize must match the image size 
                      0) # for grayscale 

bar = ProgressBar(maxval=progressbar.UnknownLength)

for i in bar(range(len(images_array))):
    # bckg substraction
    im = Image.fromarray(images_array[i])
    no_bckg = ImageChops.difference(im, bckg_im)
    out.write(np.array(no_bckg, dtype=np.uint8))
    
#out.release()
print('video post-processing is done')

print('calculation of pipette properties...')

# convert image to black and white for better pipette contour detection 

_, bckg_arr_binary=cv2.threshold(src=bckg_arr, thresh=threshold, maxval=255, type=cv2.THRESH_BINARY)
Image.fromarray(bckg_arr_binary).save(output_path + "/" + video_name + "binary_control_pipette_detection.pdf")
%matplotlib
Image.fromarray(bckg_arr_binary).show()

# extract pipette contours 
upper_contour_pipette = []
lower_contour_pipette = []

for column_index in np.arange(0,np.shape(bckg_arr)[1]): #np.shape(bckg_arr)[1]
    
    signal = bckg_arr_binary[:,column_index] # take all rows for each column 
    
    # peaks index parameters: - distance : Required minimal horizontal distance (>= 1) in samples between neighbouring peaks. 
    #              - plateau_size: Required size of the flat top of peaks in samples
    peaks_index, peaks_dict = find_peaks(signal, height = 250, distance= 30, plateau_size = plateau_size) # detect pipette internal peaks 
    
    if(len(peaks_index) == 0): 
        break
        
    # MIGHT SUBSITUTE WITH SOME CLUSTERING? (divide in two groups based on distance)
    upper_indexes = peaks_index[peaks_index < np.shape(bckg_im)[0]/2]  # assuming that the upper pipette is in the upper half of the image...  
    lower_indexes = peaks_index[peaks_index >= np.shape(bckg_im)[0]/2]  #
    
    # upper pipette
    if(len(upper_indexes) ==2 ): # if there are two peaks no problem: take internal part of pipette (the right edge of first peak abd left edge of second peak)
        upper_contour_pipette.append( (column_index, peaks_dict['right_edges'][np.where(peaks_index == upper_indexes[0])][0]) ) 
        upper_contour_pipette.append( (column_index, peaks_dict['left_edges'][np.where(peaks_index == upper_indexes[1])][0]) ) 
 
      
    # lower pipette 
    if(len(lower_indexes) ==2 ): # if there are two peaks no problem: take internal part of pipette
        lower_contour_pipette.append( (column_index, peaks_dict['right_edges'][np.where(peaks_index == lower_indexes[0])][0]) ) 
        lower_contour_pipette.append( (column_index, peaks_dict['left_edges'][np.where(peaks_index == lower_indexes[1])][0]) ) 

# do a liner fit on pipette contours 

def func(x, a, b): # linear fit function 
    return a*x+b

# defne arrays to fit 
x_up = np.array([i[0] for i in upper_contour_pipette[::2]]) # x-coordinate of the upper pipette
x_low = np.array([i[0] for i in lower_contour_pipette[::2]]) 

y1 = np.array([i[1] for i in upper_contour_pipette[::2]]) # upper contour of the upper pipette
y2 = np.array([i[1] for i in upper_contour_pipette[1::2]]) # lower contour of the upper pipette 

y3 = np.array([i[1] for i in lower_contour_pipette[::2]]) # upper contour of the lower pipette
y4 = np.array([i[1] for i in lower_contour_pipette[1::2]]) # lower contour of the lower pipette 

# do the fit 
fit_param_y1,_ =  curve_fit(func, x_up, y1)
fit_param_y2,_ =  curve_fit(func, x_up, y2)
fit_param_y3,_ =  curve_fit(func, x_low, y3)
fit_param_y4,_ =  curve_fit(func, x_low, y4)

# complete possible gaps in the x
x_up = np.arange(x_up[0], x_up[-1], 1 )
x_low = np.arange(x_low[0], x_low[-1], 1 )

# fit the pipette profile
y1_fit = fit_param_y1[0]*x_up + fit_param_y1[1] # upper contour of the upper pipette fitted
y2_fit = fit_param_y2[0]*x_up + fit_param_y2[1] # lower contour of the upper pipette fitted
y3_fit = fit_param_y3[0]*x_low + fit_param_y3[1] # upper contour of the lower pipette fitted
y4_fit = fit_param_y4[0]*x_low + fit_param_y4[1] # lower contour of the lower pipette fitted 

# save the contour line in the correct order for the pipette area to be closed :
# upper left corner --> upper right corner --> lower right corner --> lower left corner

all_upper_contour = np.append(y1_fit, y2_fit[::-1]) # upper line + reversed lower line 
all_lower_contour = np.append(y3_fit, y4_fit[::-1])

# write it in a format so that the Path function can read it ( = as list of tuples)
all_upper_contour= sorted(list(set(zip(np.append(x_up,x_up[::-1]), all_upper_contour))))
all_lower_contour = sorted(list(set(zip(np.append(x_low,x_low[::-1]), all_lower_contour))))

# find center coordinates 
# do a linear fit of the central value of y 
y_middle_up = (y1_fit + y2_fit)/2
fit_param_center_up,_ =  curve_fit(func, x_up, y_middle_up)
y_fit_center_up = fit_param_center_up[0]*x_up + fit_param_center_up[1]

y_middle_low = (y3_fit + y4_fit)/2
fit_param_center_low,_ =  curve_fit(func, x_low, y_middle_low)
y_fit_center_low = fit_param_center_low[0]*x_low + fit_param_center_low[1]

# upper pipette
x_center_up = x_up[-1]
y_center_up = y_fit_center_up[-1]

# lower pipette
x_center_low = x_low[-1]
y_center_low = y_fit_center_low[-1]

# calculate inclination angle of the pipettes 
# remember that in the image, the y axis points down (the zero is in the upper left corner of the image)

#theta_up = np.arctan((y1_fit[-1] - y1_fit[0])/(x_up[-1] - x_up[0])) 
theta_up = (fit_param_y1[0] + fit_param_y2[0])/2
theta_up = -theta_up # because I want the new y axis to point up when I do the rotation
#theta_low = np.arctan((y3_fit[-1] - y3_fit[0])/(x_low[-1] - x_low[0]))
theta_low = (fit_param_y3[0] + fit_param_y4[0])/2
theta_low = - theta_low

# Write pipette properties in a dictionary 
pipette_properties_dictionary = {
                                'upper_contour' : all_upper_contour, 
                                'lower_contour' : all_lower_contour, 
                                'x_center_upper_pipette' : x_center_up,
                                'y_center_upper_pipette': y_center_up, 
                                'theta_up' : theta_up, # inclination of the upper pipette
                                'x_center_lower_pipette': x_center_low, 
                                'y_center_lower_pipette': y_center_low,
                                'theta_low': theta_low # inclination of the lower pipette
}

# Save the pipette properties dictionary 
my_file = open(output_path+"/pipette_properties_"+video_name+".txt", "wb")
pickle.dump(pipette_properties_dictionary, my_file)
my_file.close()

# Save image to check if pipette properties are all right

# draw the x-line of the new axis centerd in the pipette center to check the rotation when right
y_test_up = y_center_up - x_center_up*np.tan(-theta_up)
test_up = [(x_center_up, y_center_up), (0, y_test_up)]
y_test_low=  y_center_low - x_center_low*np.tan(-theta_low)
test_low = [(x_center_low, y_center_low), (0, y_test_low)]


## PLOT THE PIPETTE DETECTION PROFILE AND CENTER ON THE BCKG IMAGE

#order x up
list_x_up = [i[0] for i in pipette_properties_dictionary['upper_contour'][0::2] ]
list_x_up_2 = [i[0] for i in pipette_properties_dictionary['upper_contour'][1::2] ]
list_x_up.extend(list_x_up_2[::-1])

#order y up
list_y_up = [i[1] for i in pipette_properties_dictionary['upper_contour'][0::2] ]
list_y_up_2 = [i[1] for i in pipette_properties_dictionary['upper_contour'][1::2] ]
list_y_up.extend(list_y_up_2[::-1])

# order the x low
list_x_low = [i[0] for i in pipette_properties_dictionary['lower_contour'][0::2] ]
list_x_low_2 = [i[0] for i in pipette_properties_dictionary['lower_contour'][1::2] ]
list_x_low.extend(list_x_low_2[::-1])

# order the y up
list_y_low = [i[1] for i in pipette_properties_dictionary['lower_contour'][0::2] ]
list_y_low_2 = [i[1] for i in pipette_properties_dictionary['lower_contour'][1::2] ]
list_y_low.extend(list_y_low_2[::-1])

%matplotlib
fig = plt.figure(figsize = (13, 7))
plt.imshow(Image.fromarray(bckg_arr_binary), cmap='gray')
plt.plot(list_x_up, list_y_up, color = 'red', linewidth = 2)
plt.plot(list_x_low, list_y_low, color = 'red', linewidth = 2)
plt.scatter(int(x_center_up),y_center_up, marker = 'x', linewidths = 3,  s = 100, c = 'darkorange', zorder = 10)
plt.scatter(int(x_center_low),y_center_low,marker = 'x', linewidths = 3, s = 100, c = 'darkorange',   zorder = 10)
plt.savefig(output_path + "/" + video_name + "_control_pipette_detection.png")

Extracting images and computing background...


bckg computation is done
saving bckg subtracted images in a video...


100% |########################################################################|


video post-processing is done
calculation of pipette properties...
Using matplotlib backend: <object object at 0x000001A930C7BC90>
Using matplotlib backend: QtAgg
